# Extract relevant worksheets from ONS Excel data
---
## ONS API Website
- https://developer.ons.gov.uk/dataset/

## Checklist:
- [x] Multi-factor productivity estimates (7th July 2020)
- [x] UK business: activity, size and location (29th Sept 2020)
- [x] Regional gross value added (balanced) by industry: all NUTS level regions (19th Dec 2019)
- [x] Regional gross disposable household income: all NUTS level regions (4th June 2020)

## Note: 
- To adjust for inflation, all extracted panel and time series data use 2016 as the base year for price-related data.
- Prices in cross-sectional data, however, are given in current prices for that year.

In [1]:
import numpy as np
import pandas as pd

import tempfile
import urllib
import requests

In [2]:
# Add a custom header to urllib request
# Circumvents ONS response with HTTP 403 if user agent is Python

opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [3]:
# Function to download and save file as a temporary file

def get_excel_to_dataframe(url, worksheet, header, index_col):

    # Create temporary directory in data dir
    f = tempfile.TemporaryDirectory(dir = "data/")
    
    # Get Excel workbook
    path = f'{f.name}/data.xlsx'

    try:
        resp = urllib.request.urlretrieve(url, path)
        # Excel to DataFrame
        result = pd.read_excel(path, worksheet, header=header, index_col=index_col)
        f.cleanup()
        return result

    except:
        f.cleanup()

## 1. MFP Estimates (Date: 7th July 2020)
- Table A6 - Multi-factor productivity: annual indices and log changes (industries P and Q suppressed)

In [4]:
mfp_url = "https://www.ons.gov.uk/file?uri=%2feconomy%2feconomicoutputandproductivity%2fproductivitymeasures%2fdatasets%2fmultifactorproductivityexperimentalestimatesreferencetables%2fcurrent/mfp01unlinked03072020final1.xlsx"
mfp_data = get_excel_to_dataframe(mfp_url, 'Table A6', header=3, index_col=0)
mfp_data

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,STU,TOTAL MS
2016=100,(GVA/Combined Inputs),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1970,57.4118,184.220183,43.886764,47.509904,164.028777,105.746753,74.115579,37.942492,135.938990,58.967936,83.679284,72.379110,64.908257,54.539341,NaN,NaN,NaN,65.734804,302.272727,61.508636
1971,61.0501,180.471140,44.575163,50.336339,164.071856,109.046053,75.372651,41.691843,140.963855,59.705043,78.960943,73.173432,68.880866,58.838710,NaN,NaN,NaN,69.526248,306.742739,63.061798
1972,60.9016,148.756271,45.867882,55.212580,170.780693,105.876822,76.672241,44.788252,141.446579,60.904977,75.416667,74.064926,72.859116,62.686567,NaN,NaN,NaN,69.933350,291.501416,64.339883
1973,62.3672,161.272378,49.186422,60.253542,186.119874,102.353812,75.495893,49.591352,138.912732,64.010394,74.787443,74.192463,77.959451,67.516301,NaN,NaN,NaN,72.324455,282.884448,67.108575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,10.2117,3.920710,-1.720987,-3.552453,5.099670,2.867071,2.985334,-2.434624,-0.831727,4.262185,-3.003324,4.740665,1.628041,-1.687169,NaN,NaN,NaN,2.730555,8.291525,0.798376
2016,-5.8437,-0.856347,-0.250866,-0.603540,4.913046,1.095731,3.190505,-6.399249,-2.091955,2.016050,0.331641,2.165715,-0.707673,-3.136741,NaN,NaN,NaN,0.918208,-2.865350,0.071696
2017,2.1313,9.923017,1.002860,-5.553409,-2.833823,1.810733,0.941460,2.769820,0.863343,0.689196,1.976228,2.391429,6.747020,-0.689790,NaN,NaN,NaN,-2.188189,-2.090264,1.518520
2018,2.08058,2.435921,-0.691999,-6.877706,0.551767,-2.298036,2.935592,0.874407,2.008778,4.330445,-2.758395,-0.010482,2.299392,-0.102768,NaN,NaN,NaN,1.475190,-1.927342,0.533871


In [5]:
# Drop first row, drop last total column, drop columns of NaNs, and name index "year"
# Drop bottom half of data (bottom half contain a different log value table)

mfp_data = mfp_data.iloc[1:51, :-1]
mfp_data = mfp_data.loc[:, ~mfp_data.columns.isin(['O', 'P', 'Q'])]
mfp_data.index.name = 'year'
mfp_data

,A,B,C,D,E,F,G,H,I,J,K,L,M,N,R,STU
year,,,,,,,,,,,,,,,,
1970,57.4118,184.220183,43.886764,47.509904,164.028777,105.746753,74.115579,37.942492,135.938990,58.967936,83.679284,72.379110,64.908257,54.539341,65.734804,302.272727
1971,61.0501,180.471140,44.575163,50.336339,164.071856,109.046053,75.372651,41.691843,140.963855,59.705043,78.960943,73.173432,68.880866,58.838710,69.526248,306.742739
1972,60.9016,148.756271,45.867882,55.212580,170.780693,105.876822,76.672241,44.788252,141.446579,60.904977,75.416667,74.064926,72.859116,62.686567,69.933350,291.501416
1973,62.3672,161.272378,49.186422,60.253542,186.119874,102.353812,75.495893,49.591352,138.912732,64.010394,74.787443,74.192463,77.959451,67.516301,72.324455,282.884448
1974,64.9128,140.914800,49.123094,61.142082,171.586376,96.737943,67.694994,49.158702,128.789561,59.933500,72.282440,71.127673,75.485437,66.461710,67.791908,258.818770
1975,62.2067,151.389858,48.640974,62.858879,162.726176,94.693548,65.421288,48.778183,127.693603,59.498956,76.744895,71.892066,73.218527,64.796480,67.939789,256.775701
1976,57.8496,134.410020,50.111332,63.060093,165.825755,94.475228,67.740047,50.849800,130.439684,61.677149,76.484561,74.737177,74.027460,66.202464,72.235927,254.992658
1977,64.4592,118.889746,50.405046,65.511576,166.008018,95.431220,67.425265,52.919203,131.946384,62.662866,74.144812,77.879747,71.974180,65.471311,74.900750,261.705202
1978,68.7023,116.121222,50.850311,68.577560,170.002851,101.364458,71.579743,54.756064,139.807428,63.851618,74.931954,78.537810,72.086043,66.844660,78.325663,232.532348


In [6]:
# Export to csv

mfp_data.to_csv('data/mfp_2020.csv')

## 2. UK business: activity, size, and location (Date: 29th Sept 2020)
- Table 10 - Number of VAT and/or PAYE based enterprises by employment sizebands
- Table 11 - Number of VAT and/or PAYE based enterprises by turnover sizebands 

In [7]:
ukbusiness_url = "https://www.ons.gov.uk/file?uri=%2fbusinessindustryandtrade%2fbusiness%2factivitysizeandlocation%2fdatasets%2fukbusinessactivitysizeandlocation%2f2020/ukbusinessworkbook2020.xlsx"

In [8]:
ukbusiness_employment_data = get_excel_to_dataframe(ukbusiness_url, 'Table 10 ', header=5, index_col=0)
ukbusiness_employment_data

,Unnamed: 1,0-4,5-9,10-19,20-49,50-99,100-249,250+,Total
K02000001,United Kingdom,2156975.0,305435.0,152540.0,81635.0,27215.0,15180.0,10720.0,2749700.0
K03000001,Great Britain,2097915.0,297235.0,148360.0,79300.0,26455.0,14765.0,10490.0,2674520.0
K04000001,England and Wales,1964640.0,274145.0,136585.0,73320.0,24585.0,13770.0,9785.0,2496830.0
E92000001,England,1883505.0,260680.0,130420.0,70025.0,23620.0,13265.0,9460.0,2390975.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
Data as at March 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Source: Office for National Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Drop Nan, set multi_index, drop 'total' column

ukbusiness_employment_data = ukbusiness_employment_data.dropna()
ukbusiness_employment_data = ukbusiness_employment_data.set_index(['Unnamed: 1'], append=True)
ukbusiness_employment_data.index.names = ['NUTS_code', 'region']
ukbusiness_employment_data = ukbusiness_employment_data.iloc[:,:-1]
ukbusiness_employment_data

,,0-4,5-9,10-19,20-49,50-99,100-249,250+
NUTS_code,region,,,,,,,
K02000001,United Kingdom,2156975.0,305435.0,152540.0,81635.0,27215.0,15180.0,10720.0
K03000001,Great Britain,2097915.0,297235.0,148360.0,79300.0,26455.0,14765.0,10490.0
K04000001,England and Wales,1964640.0,274145.0,136585.0,73320.0,24585.0,13770.0,9785.0
E92000001,England,1883505.0,260680.0,130420.0,70025.0,23620.0,13265.0,9460.0
E12000001,North East,53130.0,9465.0,4660.0,2570.0,795.0,490.0,350.0
...,...,...,...,...,...,...,...,...
N09000006,Fermanagh and Omagh,7090.0,580.0,280.0,150.0,45.0,20.0,10.0
N09000007,Lisburn and Castlereagh,3735.0,620.0,310.0,200.0,45.0,30.0,20.0
N09000008,Mid and East Antrim,3930.0,560.0,250.0,125.0,40.0,20.0,10.0


In [10]:
ukbusiness_sizeband_data = get_excel_to_dataframe(ukbusiness_url, 'Table 11', header=5, index_col=0)
ukbusiness_sizeband_data

,Unnamed: 1,0-49,50-99,100-249,250-499,500-999,1000-1999,2000-4999,5000-9999,10000-49999,50000+,Total
K02000001,United Kingdom,411835.0,622655.0,901405.0,351580.0,203020.0,113960.0,78985.0,30090.0,27290.0,8880.0,2749700.0
K03000001,Great Britain,392995.0,608800.0,881710.0,342360.0,197145.0,110445.0,76605.0,29200.0,26565.0,8695.0,2674520.0
K04000001,England and Wales,367095.0,569295.0,822570.0,318560.0,183715.0,103070.0,71900.0,27470.0,24970.0,8180.0,2496825.0
E92000001,England,347275.0,545610.0,789750.0,304825.0,176180.0,99140.0,69475.0,26580.0,24180.0,7955.0,2390970.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
Data as at March 2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Source: Office for National Statistics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Drop Nan, set multi_index, drop 'total' column

ukbusiness_sizeband_data = ukbusiness_sizeband_data.dropna()
ukbusiness_sizeband_data = ukbusiness_sizeband_data.set_index(['Unnamed: 1'], append=True)
ukbusiness_sizeband_data.index.names = ['NUTS_code', 'region']
ukbusiness_sizeband_data = ukbusiness_sizeband_data.iloc[:,:-1]
ukbusiness_sizeband_data

,,0-49,50-99,100-249,250-499,500-999,1000-1999,2000-4999,5000-9999,10000-49999,50000+
NUTS_code,region,,,,,,,,,,
K02000001,United Kingdom,411835.0,622655.0,901405.0,351580.0,203020.0,113960.0,78985.0,30090.0,27290.0,8880.0
K03000001,Great Britain,392995.0,608800.0,881710.0,342360.0,197145.0,110445.0,76605.0,29200.0,26565.0,8695.0
K04000001,England and Wales,367095.0,569295.0,822570.0,318560.0,183715.0,103070.0,71900.0,27470.0,24970.0,8180.0
E92000001,England,347275.0,545610.0,789750.0,304825.0,176180.0,99140.0,69475.0,26580.0,24180.0,7955.0
E12000001,North East,8665.0,17100.0,24380.0,9625.0,5250.0,2870.0,1925.0,715.0,705.0,225.0
...,...,...,...,...,...,...,...,...,...,...,...
N09000006,Fermanagh and Omagh,3410.0,1355.0,1675.0,760.0,420.0,285.0,165.0,50.0,40.0,15.0
N09000007,Lisburn and Castlereagh,925.0,1060.0,1340.0,600.0,420.0,285.0,185.0,75.0,55.0,15.0
N09000008,Mid and East Antrim,1325.0,1005.0,1265.0,580.0,355.0,190.0,125.0,45.0,40.0,5.0


In [12]:
# Export to csv

ukbusiness_employment_data.to_csv('data/business_employment_2020.csv')
ukbusiness_sizeband_data.to_csv('data/business_sizeband_2020.csv')

## 3. Regional gross value added (balanced) by industry (19th Dec 2019)
- Table 3B - NUTS 3 chained volume measures in 2016 money value

In [13]:
regional_gva_url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fgrossvalueaddedgva%2fdatasets%2fnominalandrealregionalgrossvalueaddedbalancedbyindustry%2fcurrent/regionalgrossvalueaddedbalancedbyindustryallnutslevelregions.xlsx"
regional_gva_data = get_excel_to_dataframe(regional_gva_url, 'Table3b', header=1, index_col=0)
regional_gva_data

,Region name,SIC07,SIC07 description,1998,1999,2000,2001,2002,2003,2004,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,20183
Region code,,,,,,,,,,,,,,,,,,,,,
UKC11,Hartlepool and Stockton-on-Tees,1-9,"Agriculture, forestry and fishing; mining and ...",6.0,4.0,4.0,3.0,4.0,5.0,6.0,...,12.0,17.0,43.0,48.0,61.0,59.0,53.0,25.0,31.0,35.0
UKC11,Hartlepool and Stockton-on-Tees,AB,"Agriculture, forestry and fishing; mining and ...",6.0,4.0,4.0,3.0,4.0,5.0,6.0,...,12.0,17.0,43.0,48.0,61.0,59.0,53.0,25.0,31.0,35.0
UKC11,Hartlepool and Stockton-on-Tees,10-12,"Manufacture of food, beverages and tobacco",128.0,132.0,131.0,121.0,116.0,120.0,118.0,...,110.0,127.0,125.0,123.0,83.0,76.0,63.0,112.0,108.0,120.0
UKC11,Hartlepool and Stockton-on-Tees,CA,"Manufacture of food, beverages and tobacco",128.0,132.0,131.0,121.0,116.0,120.0,118.0,...,110.0,127.0,125.0,123.0,83.0,76.0,63.0,112.0,108.0,120.0
UKC11,Hartlepool and Stockton-on-Tees,13-15,"Manufacture of textiles, wearing apparel and l...",16.0,15.0,14.0,13.0,11.0,13.0,10.0,...,7.0,9.0,8.0,6.0,5.0,6.0,6.0,4.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UKN16,Fermanagh and Omagh,Total,All industries,1495.0,1578.0,1654.0,1699.0,1722.0,1823.0,1896.0,...,1895.0,1860.0,1858.0,1890.0,1898.0,1952.0,1968.0,2046.0,2162.0,2094.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Workplace-based estimates are allocated to the region in which the economic activity takes place.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# Drop NaN
# Rename last column from '20183' to '2018'
# Set multindex

regional_gva_data = regional_gva_data.dropna()
regional_gva_data = regional_gva_data.rename(columns={20183: 2018})
regional_gva_data = regional_gva_data.set_index(['Region name', 'SIC07', 'SIC07 description'], append=True)
regional_gva_data.index.names = ['NUTS_code', 'region', 'SIC07', 'SIC07_desc']
regional_gva_data

1998  \
NUTS_code region                          SIC07 SIC07_desc                                                   
UKC11     Hartlepool and Stockton-on-Tees 1-9   Agriculture, forestry and fishing; mining and q...     6.0   
                                          AB    Agriculture, forestry and fishing; mining and q...     6.0   
                                          10-12 Manufacture of food, beverages and tobacco           128.0   
                                          CA    Manufacture of food, beverages and tobacco           128.0   
                                          13-15 Manufacture of textiles, wearing apparel and le...    16.0   
...                                                                                                    ...   
UKN16     Fermanagh and Omagh             S     Other service activities                              14.0   
                                          97-98 Households as employers and own use production         3.0   
                                          T     Activities of households                               3.0   
                                          G-T   Services sector                                     1043.0   
                                          Total All industries                                      1495.0   

                                                                                                      1999  \
NUTS_code region                          SIC07 SIC07_desc                                                   
UKC11     Hartlepool and Stockton-on-Tees 1-9   Agriculture, forestry and fishing; mining and q...     4.0   
                                          AB    Agriculture, forestry and fishing; mining and q...     4.0   
                                          10-12 Manufacture of food, beverages and tobacco           132.0   
                                          CA    Manufacture of food, beverages and tobacco           132.0   
                                          13-15 Manufacture of textiles, wearing apparel and le...    15.0   
...                                                                                                    ...   
UKN16     Fermanagh and Omagh             S     Other service activities                              14.0   
                                          97-98 Households as employers and own use production         4.0   
                                          T     Activities of households                               4.0   
                                          G-T   Services sector                                     1088.0   
                                          Total All industries                                      1578.0   

                                                                                                      2000  \
NUTS_code region                          SIC07 SIC07_desc                                                   
UKC11     Hartlepool and Stockton-on-Tees 1-9   Agriculture, forestry and fishing; mining and q...     4.0   
                                          AB    Agriculture, forestry and fishing; mining and q...     4.0   
                                          10-12 Manufacture of food, beverages and tobacco           131.0   
                                          CA    Manufacture of food, beverages and tobacco           131.0   
                                          13-15 Manufacture of textiles, wearing apparel and le...    14.0   
...                                                                                                    ...   
UKN16     Fermanagh and Omagh             S     Other service activities                              14.0   
                                          97-98 Households as employers and own use production         5.0   
                                          T     Activities of households                               5.0   
                              

In [15]:
# Export to csv

regional_gva_data.to_csv('data/regional_gva_2020.csv')

## 4. Regional gross disposable household income: all NUTS level regions (4th June 2020)
- Table 7 - Components of GDHI per head of population at current basic prices

In [16]:
regional_gdhi_url = "https://www.ons.gov.uk/file?uri=%2feconomy%2fregionalaccounts%2fgrossdisposablehouseholdincome%2fdatasets%2fregionalgrossdisposablehouseholdincomegdhi%2f1997to2018/regionalgrossdisposablehouseholdincomeallnutslevelregions.xls"
regional_gdhi_data = get_excel_to_dataframe(regional_gdhi_url, 'Table 7', header=1, index_col=0)
regional_gdhi_data

,NUTS code,Region name,Transaction code,Transaction,1997,1998,1999,2000,2001,2002,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,20181
NUTS level,,,,,,,,,,,,,,,,,,,,,
UK,UK,United Kingdom,B.2g,Operating surplus,1763.0,1815.0,1909.0,1969.0,2030.0,2038.0,...,2037.0,2017.0,2077.0,2212.0,2222.0,2383.0,2517.0,2645.0,2632.0,2717.0
UK,UK,United Kingdom,B.3g,Mixed income,999.0,1061.0,1119.0,1168.0,1222.0,1302.0,...,1544.0,1598.0,1632.0,1686.0,1754.0,1856.0,1934.0,1980.0,2068.0,2215.0
UK,UK,United Kingdom,D.1,Compensation of employees,7534.0,7932.0,8537.0,9108.0,9612.0,9885.0,...,12664.0,12916.0,13075.0,13232.0,13734.0,13997.0,14273.0,14736.0,15276.0,15844.0
UK,UK,United Kingdom,D.4,"Property income, received",2589.0,2696.0,2470.0,2869.0,2710.0,2689.0,...,3132.0,3093.0,3019.0,2911.0,2978.0,3346.0,3586.0,3396.0,3204.0,3536.0
UK,UK,United Kingdom,NaN,Primary resources total,12885.0,13503.0,14035.0,15114.0,15575.0,15914.0,...,19376.0,19625.0,19802.0,20042.0,20689.0,21582.0,22310.0,22757.0,23181.0,24312.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NUTS3,UKN16,Fermanagh and Omagh,NaN,Secondary uses total,3136.0,3474.0,3525.0,3843.0,3916.0,4217.0,...,5492.0,6041.0,6121.0,6152.0,5956.0,5956.0,6203.0,6268.0,6414.0,6430.0
NUTS3,UKN16,Fermanagh and Omagh,NaN,Balance of secondary income,-657.0,-820.0,-922.0,-1174.0,-1315.0,-1393.0,...,-1154.0,-1235.0,-1182.0,-907.0,-748.0,-805.0,-903.0,-736.0,-758.0,-690.0
NUTS3,UKN16,Fermanagh and Omagh,B.6g,Gross Disposable Income,7649.0,7716.0,8111.0,8433.0,8688.0,9141.0,...,12429.0,12641.0,13027.0,13224.0,13481.0,13822.0,14685.0,15232.0,15916.0,16569.0


In [17]:
# Drop NaN
# Rename last column from '20183' to '2018'
# Set multindex

regional_gdhi_data = regional_gdhi_data.dropna()
regional_gdhi_data = regional_gdhi_data.rename(columns={'20181': '2018'})
regional_gdhi_data = regional_gdhi_data.set_index(['NUTS code', 'Region name', 'Transaction code', 'Transaction'], append=True)
regional_gdhi_data.index.names = ['NUTS_level', 'NUTS_code', 'region', 'transaction_code', 'transaction_desc']
regional_gdhi_data

1997  \
NUTS_level NUTS_code region              transaction_code transaction_desc                                    
UK         UK        United Kingdom      B.2g             Operating surplus                          1763.0   
                                         B.3g             Mixed income                                999.0   
                                         D.1              Compensation of employees                  7534.0   
                                         D.4              Property income, received                  2589.0   
                                                          Property income, paid                       561.0   
...                                                                                                     ...   
NUTS3      UKN16     Fermanagh and Omagh D.7              Other current transfers, received           421.0   
                                         D.5              Current taxes on income, wealth etc         799.0   
                                         D.61/D.62        Social contributions/Social benefits paid  1863.0   
                                         D.7              Other current transfers, paid               474.0   
                                         B.6g             Gross Disposable Income                    7649.0   

                                                                                                       1998  \
NUTS_level NUTS_code region              transaction_code transaction_desc                                    
UK         UK        United Kingdom      B.2g             Operating surplus                          1815.0   
                                         B.3g             Mixed income                               1061.0   
                                         D.1              Compensation of employees                  7932.0   
                                         D.4              Property income, received                  2696.0   
                                                          Property income, paid                       619.0   
...                                                                                                     ...   
NUTS3      UKN16     Fermanagh and Omagh D.7              Other current transfers, received           527.0   
                                         D.5              Current taxes on income, wealth etc         911.0   
                                         D.61/D.62        Social contributions/Social benefits paid  1972.0   
                                         D.7              Other current transfers, paid               590.0   
                                         B.6g             Gross Disposable Income                    7716.0   

                                                                                                       1999  \
NUTS_level NUTS_code region              transaction_code transaction_desc                                    
UK         UK        United Kingdom      B.2g             Operating surplus                          1909.0   
                                         B.3g             Mixed income                               1119.0   
                                         D.1              Compensation of employees                  8537.0   
                                         D.4              Property income, received                  2470.0   
                                                          Property income, paid                       582.0   
...                                                                                                     ...   
NUTS3      UKN16     Fermanagh and Omagh D.7              Other current transfers, received           520.0   
                                         D.5              Current taxes on income, wealth etc         999.0   
                                         D.61/D.62        Social contributions/Social benefits paid  1944

In [18]:
# Export to csv

regional_gdhi_data.to_csv('data/regional_gdhi_2020.csv')